In [1]:

"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/spoken_language_understanding/atis/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-beta1

In [3]:
import tensorflow as tf
import pprint
import logging
import time
import numpy as np

from sklearn.metrics import classification_report

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-beta1
GPU Enabled: True


In [0]:
def get_vocab(vocab_path):
  word2idx = {}
  with open(vocab_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
def data_generator(f_path, params):
  print('Reading', f_path)
  with open(f_path) as f:
    for line in f:
      line = line.rstrip()
      text, slot_intent = line.split('\t')
      words = text.split()[1:-1]
      slot_intent = slot_intent.split()
      slots, intent = slot_intent[1:-1], slot_intent[-1]
      assert len(words) == len(slots)
      
      words = [params['word2idx'].get(w, len(params['word2idx'])) for w in words]
      intent = params['intent2idx'].get(intent, len(params['intent2idx']))
      slots = [params['slot2idx'].get(s, len(params['slot2idx'])) for s in slots]
      
      yield (words, (intent, slots))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ((), [None]))
  _types = (tf.int32, (tf.int32, tf.int32))
  _pads = (0, (-1, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(1, _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def get_timing_signal_1d(length,
                         channels,
                         min_timescale=1.0,
                         max_timescale=1.0e4,
                         start_index=0):
  to_float = lambda x: tf.cast(x, tf.float32)
  position = to_float(tf.range(length) + start_index)
  num_timescales = channels // 2
  log_timescale_increment = (
      tf.math.log(float(max_timescale) / float(min_timescale)) /
      tf.maximum(to_float(num_timescales) - 1, 1))
  inv_timescales = min_timescale * tf.exp(
      to_float(tf.range(num_timescales)) * -log_timescale_increment)
  scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
  signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
  signal = tf.pad(signal, [[0, 0], [0, tf.compat.v1.mod(channels, 2)]])
  signal = tf.reshape(signal, [1, length, channels])
  return signal

In [0]:
class LayerNorm(tf.keras.layers.Layer):
  def __init__(self, params):
    super().__init__()
    self._epsilon = params['epsilon']
    self._hidden_units = params['global_units']
  
  def build(self, input_shape):
    self.scale = self.add_weight(name='scale',
                                 shape=[self._hidden_units],
                                 initializer=tf.ones_initializer(),
                                 trainable=True)
    self.bias = self.add_weight(name='bias',
                                shape=[self._hidden_units],
                                initializer=tf.zeros_initializer(),
                                trainable=True)
    super().build(input_shape)
  
  def call(self, inputs):
    mean, variance = tf.nn.moments(inputs, [-1], keepdims=True)
    norm_x = (inputs - mean) * tf.math.rsqrt(variance + self._epsilon)
    return norm_x * self.scale + self.bias
  
  def compute_output_shape(self, input_shape):
    return input_shape


class EncoderBlock(tf.keras.Model):
  def __init__(self, SubModel, params, name):
    super().__init__(name = name)
    self.layer_norm = LayerNorm(params)
    self.sub_model = SubModel(params)
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
  
  def call(self, inputs, training=False):
    inputs, masks = inputs
    x = self.layer_norm(inputs)
    x = self.sub_model((x, masks), training=training)
    x = self.dropout(x, training=training)
    x += inputs
    return x


class MultiheadSelfAttention(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.qkv_linear = tf.keras.layers.Dense(3 * params['global_units'], name='qkv_linear')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(params['global_units'], name='out_linear')
    self._num_heads = params['num_heads']
    self._is_bidirectional = params['is_bidirectional']
  
  def call(self, inputs, training=False):
    x, masks = inputs
    timesteps = tf.shape(x)[1]
    
    q_k_v = self.qkv_linear(x)
    q, k, v = tf.split(q_k_v, 3, axis=-1)
    
    if self._num_heads > 1:
      q = tf.concat(tf.split(q, self._num_heads, axis=2), axis=0)                        
      k = tf.concat(tf.split(k, self._num_heads, axis=2), axis=0)                        
      v = tf.concat(tf.split(v, self._num_heads, axis=2), axis=0)
    
    align = tf.matmul(q, k, transpose_b=True)
    align *= tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))
    
    if (masks is not None) or (not self._is_bidirectional):
      paddings = tf.fill(tf.shape(align), float('-inf'))
    
    if masks is not None:
      c_masks = tf.tile(masks, [params['num_heads'], 1])
      c_masks = tf.tile(tf.expand_dims(c_masks, 1), [1, timesteps, 1])
      align = tf.where(tf.equal(c_masks, 0), paddings, align)
    
    if not self._is_bidirectional:
      lower_tri = tf.ones((timesteps, timesteps))                                       
      lower_tri = tf.linalg.LinearOperatorLowerTriangular(lower_tri).to_dense()      
      t_masks = tf.tile(tf.expand_dims(lower_tri, 0), [tf.shape(align)[0], 1, 1])     
      align = tf.where(tf.equal(t_masks, 0), paddings, align)
    
    align = tf.nn.softmax(align)
    align = self.dropout(align, training=training)
    
    if masks is not None:
      q_masks = tf.tile(masks, [params['num_heads'], 1])
      q_masks = tf.tile(tf.expand_dims(q_masks, 2), [1, 1, timesteps])
      align *= tf.cast(q_masks, tf.float32)
    
    x = tf.matmul(align, v)
    if self._num_heads > 1:
      x = tf.concat(tf.split(x, self._num_heads, axis=0), axis=2)
    x = self.out_linear(x)
    
    return x
  

class PointwiseFFN(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.dense_1 = tf.keras.layers.Dense(params['multiplier']*params['global_units'], tf.nn.relu, name='fc_relu')
    self.dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.dense_2 = tf.keras.layers.Dense(params['global_units'], name='linear')
  
  def call(self, inputs, training=False):
    x, masks = inputs
    return self.dense_2(self.dropout(self.dense_1(x), training=training))

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    self.embedding = tf.Variable(np.load(params['vocab_path']),
                                 dtype=tf.float32,
                                 name='pretrained_embedding')
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.blocks = []
    for i in range(params['num_layers']):
      self.blocks.append(EncoderBlock(
          MultiheadSelfAttention, params, name='layer{}.1'.format(i+1)))
      self.blocks.append(EncoderBlock(
          PointwiseFFN, params, name='layer{}.2'.format(i+1)))
    
    self.out_linear_intent = tf.keras.layers.Dense(params['intent_size'], name='output_intent')
    self.out_linear_slot = tf.keras.layers.Dense(params['slot_size'], name='output_slot')
    
  
  def call(self, inputs, training=None):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    masks = tf.sign(inputs)
    
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    if params['is_embedding_scaled']:
      x *= tf.sqrt(tf.cast(params['global_units'], tf.float32))
    x += get_timing_signal_1d(tf.shape(x)[1], params['global_units'])
    x = self.input_dropout(x, training=training)
    
    for block in self.blocks:
      x = block((x, masks), training=training)
    
    x_intent = tf.reduce_max(x, 1)
    x_intent = self.out_linear_intent(x_intent)
    x_slot = self.out_linear_slot(x)
    return (x_intent, x_slot)

In [0]:
params = {
  'train_path': '../data/atis.train.w-intent.iob',
  'test_path': '../data/atis.test.w-intent.iob',
  'word_path': '../vocab/word.txt',
  'vocab_path': '../vocab/word.npy',
  'intent_path': '../vocab/intent.txt',
  'slot_path': '../vocab/slot.txt',
  'batch_size': 16,
  'num_samples': 4978,
  'num_layers': 2,
  'global_units': 300,
  'num_heads': 6,
  'multiplier': 2,
  'dropout_rate': 0.1,
  'epsilon': 1e-6,
  'is_bidirectional': True,
  'is_embedding_scaled': False,
  'lr': 3e-4,
  'num_patience': 5,
}

In [0]:
params['word2idx'] = get_vocab(params['word_path'])
params['intent2idx'] = get_vocab(params['intent_path'])
params['slot2idx'] = get_vocab(params['slot_path'])

params['word_size'] = len(params['word2idx']) + 1
params['intent_size'] = len(params['intent2idx']) + 1
params['slot_size'] = len(params['slot2idx']) + 1

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [13]:
model = Model(params)
model.build(input_shape=(None, None))
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.9)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for (words, (intent, slots)) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      y_intent, y_slots = model(words, training=True)
      loss_intent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intent, logits=y_intent)
      loss_intent = tf.reduce_mean(loss_intent)
      # weighted cross entropy (weight of 'O' is set to be small)
      loss_slots = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=slots, logits=y_slots)
      weights = tf.cast(tf.sign(slots), tf.float32)
      padding = tf.constant(1e-2, tf.float32, weights.shape)
      weights = tf.where(tf.equal(weights, 0.), padding, weights)
      loss_slots = tf.reduce_mean(loss_slots * weights)
      # joint loss
      loss = loss_intent + loss_slots
    
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Loss_intent: {:.4f} | Loss_slots: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), loss_intent.numpy().item(), loss_slots.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
    
  # EVALUATION
  acc_intent = tf.keras.metrics.Accuracy()
  acc_slots = tf.keras.metrics.Accuracy()
  slot_true = []
  slot_pred = []

  for (words, (intent, slots)) in dataset(is_training=False, params=params):
    y_intent, y_slots = model(words, training=False)
    y_intent = tf.argmax(y_intent, -1)
    y_slots = tf.argmax(y_slots, -1)
    
    acc_intent.update_state(y_true=intent, y_pred=y_intent)
    acc_slots.update_state(y_true=slots, y_pred=y_slots, sample_weight=tf.sign(slots))
    
    slot_true += slots.numpy().flatten().tolist()
    slot_pred += y_slots.numpy().flatten().tolist()
    
  acc_intent = acc_intent.result().numpy()
  acc_slots = acc_slots.result().numpy()
  logger.info("Evaluation: Testing Accuracy of Intent: {:.3f}".format(acc_intent))
  logger.info("Evaluation: Testing Accuracy of Slots ('O' filtered): {:.3f}".format(acc_slots))
  logger.info('\n'+classification_report(y_true = slot_true,
                                         y_pred = slot_pred,
                                         labels = list(params['slot2idx'].values()),
                                         target_names = list(params['slot2idx'].keys()),
                                         digits=3))
  history_acc.append(acc_slots)
  
  if acc_slots > best_acc:
    best_acc = acc_slots
    # you can save model here
  logger.info("Best Slot Accuracy ('O' filtered): {:.3f}".format(best_acc))
  
  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Slot Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0712 01:45:52.929208 140684995266432 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


[('layer1.1/layer_norm/scale:0', TensorShape([300])),
 ('layer1.1/layer_norm/bias:0', TensorShape([300])),
 ('layer1.1/multihead_self_attention/qkv_linear/kernel:0',
  TensorShape([300, 900])),
 ('layer1.1/multihead_self_attention/qkv_linear/bias:0', TensorShape([900])),
 ('layer1.1/multihead_self_attention/out_linear/kernel:0',
  TensorShape([300, 300])),
 ('layer1.1/multihead_self_attention/out_linear/bias:0', TensorShape([300])),
 ('layer1.2/layer_norm_1/scale:0', TensorShape([300])),
 ('layer1.2/layer_norm_1/bias:0', TensorShape([300])),
 ('layer1.2/pointwise_ffn/fc_relu/kernel:0', TensorShape([300, 600])),
 ('layer1.2/pointwise_ffn/fc_relu/bias:0', TensorShape([600])),
 ('layer1.2/pointwise_ffn/linear/kernel:0', TensorShape([600, 300])),
 ('layer1.2/pointwise_ffn/linear/bias:0', TensorShape([300])),
 ('layer2.1/layer_norm_2/scale:0', TensorShape([300])),
 ('layer2.1/layer_norm_2/bias:0', TensorShape([300])),
 ('layer2.1/multihead_self_attention_1/qkv_linear/kernel:0',
  TensorShap

I0712 01:45:54.171890 140684995266432 interactiveshell.py:2882] Step 0 | Loss: 7.3004 | Loss_intent: 6.0999 | Loss_slots: 1.2005 | Spent: 1.2 secs | LR: 0.000300
I0712 01:45:58.660513 140684995266432 interactiveshell.py:2882] Step 50 | Loss: 1.2374 | Loss_intent: 0.6367 | Loss_slots: 0.6007 | Spent: 4.5 secs | LR: 0.000298
I0712 01:46:02.890835 140684995266432 interactiveshell.py:2882] Step 100 | Loss: 1.0993 | Loss_intent: 0.5186 | Loss_slots: 0.5807 | Spent: 4.2 secs | LR: 0.000297
I0712 01:46:07.103268 140684995266432 interactiveshell.py:2882] Step 150 | Loss: 0.9069 | Loss_intent: 0.4697 | Loss_slots: 0.4372 | Spent: 4.2 secs | LR: 0.000295
I0712 01:46:11.263034 140684995266432 interactiveshell.py:2882] Step 200 | Loss: 1.0228 | Loss_intent: 0.7431 | Loss_slots: 0.2797 | Spent: 4.2 secs | LR: 0.000294
I0712 01:46:15.451846 140684995266432 interactiveshell.py:2882] Step 250 | Loss: 0.6032 | Loss_intent: 0.0539 | Loss_slots: 0.5493 | Spent: 4.2 secs | LR: 0.000292
I0712 01:46:19.6305

Reading ../data/atis.test.w-intent.iob


I0712 01:46:44.028232 140684995266432 interactiveshell.py:2882] Evaluation: Testing Accuracy of Intent: 0.880
I0712 01:46:44.029216 140684995266432 interactiveshell.py:2882] Evaluation: Testing Accuracy of Slots ('O' filtered): 0.626
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
I0712 01:46:44.057873 140684995266432 interactiveshell.py:2882] 
                              precision    recall  f1-score   support

                           O      0.985     0.407     0.576      5501
           B-toloc.city_name      0.570     0.866     0.688       716
         B-fro

...

Reading ../data/atis.test.w-intent.iob


I0712 03:22:57.273678 140684995266432 interactiveshell.py:2882] Evaluation: Testing Accuracy of Intent: 0.970
I0712 03:22:57.274611 140684995266432 interactiveshell.py:2882] Evaluation: Testing Accuracy of Slots ('O' filtered): 0.952
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
I0712 03:22:57.301791 140684995266432 interactiveshell.py:2882] 
                              precision    recall  f1-score   support

                           O      0.999     0.979     0.989      5501
           B-toloc.city_name      0.979     0.996     0.988       716
         B-fro

Reading ../data/atis.train.w-intent.iob


I0712 03:22:59.835674 140684995266432 interactiveshell.py:2882] Step 35900 | Loss: 0.0034 | Loss_intent: 0.0000 | Loss_slots: 0.0034 | Spent: 28.8 secs | LR: 0.000007
I0712 03:23:04.280714 140684995266432 interactiveshell.py:2882] Step 35950 | Loss: 0.0002 | Loss_intent: 0.0000 | Loss_slots: 0.0002 | Spent: 4.4 secs | LR: 0.000007


KeyboardInterrupt: ignored